In [6]:
import pandas as pd
import rasterio
from rasterio.transform import from_origin
import numpy as np

In [5]:
# Reading the 1.4GB LAT/LONG CSV (46 Million Rows Long)
df = pd.read_csv('possible_tif.csv').drop(columns=['Unnamed: 0'])
df

,latitude,longitude,topografia
0,-52.5250,-28.9750,571.0
1,-52.5242,-28.9750,547.0
2,-52.5233,-28.9750,529.0
3,-52.5225,-28.9750,542.0
4,-52.5217,-28.9750,566.0
...,...,...,...
9374395,-49.4792,-31.0241,0.0
9374396,-49.4783,-31.0241,0.0
9374397,-49.4775,-31.0241,0.0
9374398,-49.4767,-31.0241,0.0


In [8]:
latitude = df['latitude'].values
longitude = df['longitude'].values
topografia = df['topografia'].values

lat_uniq = np.unique(latitude)
lon_uniq = np.unique(longitude)

grid = np.empty((len(lon_uniq), len(lat_uniq)))

grid[:] = np.nan

In [26]:
for i, (lat, lon, val) in enumerate(zip(latitude, longitude, topografia)):
    row = np.where(lat_uniq == lat)[0][0]
    col = np.where(lon_uniq == lon)[0][0]

    grid[col, row].append(val)

AttributeError: 'numpy.float64' object has no attribute 'append'

In [23]:
res_lon = lon_uniq[2] - lon_uniq[1]
res_lat = lat_uniq[2] - lat_uniq[1]

x_min = lon_uniq.max() - res_lon / 2
y_min = lat_uniq.max() - res_lat / 2

In [24]:
transform = from_origin(y_min, x_min, res_lat, res_lon)

In [25]:
with rasterio.open(
    'topografia.tif',
    'w',
    driver='GTiff',
    height=grid.shape[0],
    width=grid.shape[1],
    count=1,
    dtype=grid.dtype,
    crs='EPSG:4326',
    transform=transform
) as dst:
    dst.write(grid, 1)